In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Polygon

from keypoint_matching import KeypointMatcher
from RANSAC import ImageAlignment, project_2d_to_6d, project_1d_to_2d
from utils import show_two_images

In [ ]:
# read & show images
left = cv2.imread('left.jpg')
left = cv2.cvtColor(left, cv2.COLOR_BGR2RGB)
left_gray = cv2.cvtColor(left, cv2.COLOR_RGB2GRAY)
right = cv2.imread('right.jpg')
right = cv2.cvtColor(right, cv2.COLOR_BGR2RGB)
right_gray = cv2.cvtColor(right, cv2.COLOR_RGB2GRAY)
show_two_images(left, right, title="Given pair of images.")

In [ ]:
# get matches
kp_matcher = KeypointMatcher()
matches, kp1, des1, kp2, des2 = kp_matcher.match(left_gray, right_gray, show_matches=True)

In [ ]:
image_alignment = ImageAlignment()
best_params = image_alignment.align(
    left_gray, kp1, right_gray, kp2, matches, show_warped_image=True, num_matches=4, max_iter=2000
)

In [ ]:
# try transforming image 2 into image 1

In [ ]:
AM = np.zeros((3, 3))
AM[0, :2] = best_params[:2]
AM[1, :2] = best_params[2:4]
AM[0, 2] = best_params[4]
AM[1, 2] = best_params[5]
AM[-1, -1] = 1.0

In [ ]:
AM

In [ ]:
np.linalg.inv(AM)

In [ ]:
M_inv = np.linalg.inv(AM)[:2]

In [ ]:
M_inv

In [ ]:
rows, cols = right_gray.shape[:2]
img2_warped = cv2.warpAffine(right_gray, M_inv, (cols, rows))

In [ ]:
show_two_images(right_gray, img2_warped)

Compute corners of `right` in the co-ordinate space of `left`

In [ ]:
best_params_2_to_1 = np.array([M_inv[0, 0], M_inv[0, 1], M_inv[1, 0], M_inv[1, 1], M_inv[0, 2], M_inv[1, 2]])

In [ ]:
H1, W1 = left_gray.shape
C1 = np.array([[0, 0], [W1, 0], [0, H1], [W1, H1]])

In [ ]:
H2, W2 = right_gray.shape

C2 = np.array([[0, 0], [W2, 0], [0, H2], [W2, H2]])
C2_viewed_from_1 = np.dot(project_2d_to_6d(C2), best_params_2_to_1)
C2_viewed_from_1 = project_1d_to_2d(C2_viewed_from_1)

In [ ]:
C2

In [ ]:
C2_viewed_from_1

#### Show canvas containing both images and Transformed(`right`)

In [ ]:

H, W = 400, 800
canvas = np.zeros((H, W, 3))
canvas[:H1, :W1, :] = left

canvas[:H2, W - W2:, :] = right

In [ ]:
canvas_copy = np.zeros((H, W, 3))
canvas_copy[:H2, :W2, :] = right
canvas_copy = cv2.warpAffine(canvas_copy, M_inv, (W, H))
plt.imshow(canvas_copy.astype(int))
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(13, 7), dpi=100)

ax.imshow(canvas.astype(int), alpha=0.6)
ax.imshow(canvas_copy.astype(int), alpha=0.4)
ax.scatter(C2_viewed_from_1[:, 0], C2_viewed_from_1[:, 1], c="red", label="Corners of Image 2 transformed")

title = "Image 1 (Left), Image 2 (Right) and Image 2 transformed (middle) based on estimated affine parameters."
ax.set_title(title, fontsize=13)

plt.legend(loc="lower right")
plt.show()

#### Estimate the size of stiched image

In [ ]:
C2_viewed_from_1

In [ ]:
H_new, W_new = None, None

Y_top = np.max([0, C2_viewed_from_1[0, 1], C2_viewed_from_1[1, 1]])
Y_bot = np.min([H1, C2_viewed_from_1[2, 1], C2_viewed_from_1[3, 1]])
X_left = np.min([0, C2_viewed_from_1[0, 0], C2_viewed_from_1[3, 0]])
X_right = np.max([W1, C2_viewed_from_1[1, 0], C2_viewed_from_1[2, 0]])

H_new = Y_bot - Y_top
W_new = X_right - X_left

In [ ]:
Y_top, Y_bot, X_left, X_right, \
(H_new, W_new)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(13, 7), dpi=100)

ax.imshow(canvas.astype(int), alpha=0.6)
ax.imshow(canvas_copy.astype(int), alpha=0.4)
ax.scatter(C2_viewed_from_1[:, 0], C2_viewed_from_1[:, 1], c="red", label="Corners of Image 2 transformed")

stiched_image_rect = Rectangle(
    (X_left, Y_top), W_new, H_new, fc='none', ec='limegreen', lw=1, label="Border of stiched image",
)
ax.add_patch(stiched_image_rect)

title = "Image 1 (Left), Image 2 (Right) and Image 2 transformed (middle) based on estimated affine parameters."
ax.set_title(title, fontsize=13)

plt.legend(loc="lower right")
plt.savefig("./results/LR_intermediate.png", bbox_inches="tight")
plt.show()

In [ ]:
SEED = np.random.get_state()[1][0]

#### Fill the stiched image pixel-by-pixel

In [ ]:
I = np.meshgrid(np.arange(int(X_left), int(X_right), 1), np.arange(int(Y_top), int(Y_bot), 1))[0]

In [ ]:
I.shape

In [ ]:
int(H_new), int(W_new)

In [ ]:
assert I.shape == (int(H_new), int(W_new))

In [ ]:
pixels = []
for x in np.arange(int(X_left), int(X_right), 1):
    for y in np.arange(int(Y_top), int(Y_bot), 1):
        pixels.append([y, x])

In [ ]:
len(pixels)

In [ ]:
def check_point_inside_rectangle(point, rect_top_left, rect_bot_right):

    assert isinstance(point, (tuple, list, np.ndarray))
    assert len(point) == 2

    assert isinstance(rect_top_left, (tuple, list, np.ndarray))
    assert len(point) == 2

    assert isinstance(rect_bot_right, (tuple, list, np.ndarray))
    assert len(point) == 2

    x, y = point
    x1, y1 = rect_top_left
    x2, y2 = rect_bot_right
    
    return ((x <= x2) and (x > x1)) and ((y <= y2) and (y > y1))

In [ ]:
check_point_inside_rectangle((0.5, 0.5), (0, 0), (1, 1)), check_point_inside_rectangle((0.5, -0.5), (0, 0), (1, 1))

In [ ]:
C2_viewed_from_1

In [ ]:
C1

In [ ]:
C2_viewed_from_1

In [ ]:
C2_viewed_from_1_reordered = np.array(
    [C2_viewed_from_1[0], C2_viewed_from_1[1], C2_viewed_from_1[3], C2_viewed_from_1[2]]
)

In [ ]:
P2 = Polygon(C2_viewed_from_1_reordered)

In [ ]:
C1_reordered = np.array([C1[0], C1[1], C1[3], C1[2]])

In [ ]:
P2.contains_point((300, 300))

In [ ]:
P1 = Polygon(C1_reordered)

In [ ]:
P1.contains_point((500, 100))

In [ ]:
C1

In [ ]:
C1_reordered

In [ ]:
C2_viewed_from_1_reordered

In [ ]:
C1_reordered = np.array([C1[0], C1[1], C1[3], C1[2]])
P1 = Polygon(C1_reordered, color="red")

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
rect = plt.Rectangle((0.2, 0.75), 0.4, 0.15, color='k', alpha=0.3)
circ = plt.Circle((0.7, 0.2), 0.15, color='b', alpha=0.3)
pgon = plt.Polygon([[0.15, 0.15], [0.35, 0.4], [0.2, 0.6]],
color='g', alpha=0.5)

ax.add_patch(rect)
ax.add_patch(circ)
ax.add_patch(pgon)
ax.add_patch(P1)
ax.set_xlim(0, 1000)
ax.set_ylim(0, 1000)


In [ ]:
merged_image = np.zeros(canvas.shape)
merging_weights_1 = np.zeros(canvas.shape)
merging_weights_2 = np.zeros(canvas.shape)
checker = np.zeros(canvas.shape)

C1_reordered = np.array([C1[0], C1[1], C1[3], C1[2]])
P1 = Polygon(C1_reordered)

C2_viewed_from_1_reordered = np.array(
    [C2_viewed_from_1[0], C2_viewed_from_1[1], C2_viewed_from_1[3], C2_viewed_from_1[2]]
)
P2 = Polygon(C2_viewed_from_1_reordered)

for x in np.arange(int(X_left), int(X_right), 1):
    for y in np.arange(int(Y_top), int(Y_bot), 1):
        
        in_img_1 = P1.contains_point((x, y))
        in_img_2 = P2.contains_point((x, y))
        
        if in_img_1 and not in_img_2:
            merging_weights_1[y, x] = 1.0
            merging_weights_2[y, x] = 0.0

        if in_img_2 and not in_img_1:
            merging_weights_1[y, x] = 0.0
            merging_weights_2[y, x] = 1.0

        if in_img_1 and in_img_2:
            merging_weights_1[y, x] = 0.5
            merging_weights_2[y, x] = 0.5
            checker[y, x] = 1.0

In [ ]:
plt.imshow(merging_weights_1)

In [ ]:
plt.imshow(merging_weights_2)

In [ ]:
canvas.shape, canvas_copy.shape

In [ ]:
merging_weights_1.shape, merging_weights_2.shape

In [ ]:
canvas_final = np.multiply(canvas, merging_weights_1) + np.multiply(canvas_copy, merging_weights_2)

In [ ]:
canvas_final_cropped = canvas_final[int(Y_top):int(Y_bot), int(X_left):int(X_right)]

In [ ]:
# generate some data
x = np.arange(0, 10, 0.2)
y = np.sin(x)

# plot it
f, ax = plt.subplots(1, 3, gridspec_kw={'width_ratios': [2, 2, 4]}, figsize=(20, 6))

ax[0].imshow(left)
ax[0].set_title("Left image", fontsize=18)
ax[0].axis("off")

ax[1].imshow(right)
ax[1].set_title("Right image", fontsize=18)
ax[1].axis("off")

ax[2].imshow(canvas_final_cropped.astype(int))
ax[2].set_title("Stitched image", fontsize=18)
ax[2].axis("off")

f.tight_layout()
f.savefig('./results/all_stiched.png', bbox_inches="tight")